In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import pandas as pd
from functions import main_cleaning

pd.set_option('display.max_columns', None) 

df_raw = pd.read_csv("dielectron.csv")

In [19]:
# both_diff = (((df_raw["Q1"] == -1) & (df_raw["Q2"] == 1) ) | ((df_raw["Q1"] == 1) & (df_raw["Q2"] == -1))).sum()
# both_neg = ((df_raw["Q1"] == -1) & (df_raw["Q2"] == -1)).sum()
# both_pos = ((df_raw["Q1"] == 1) & (df_raw["Q2"] == 1)).sum()

# both_diff, both_neg, both_pos

In [20]:
df_cleaned = main_cleaning(df_raw)

e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_same_charge"] = df["Q1"] == df["Q2"]


In [36]:
df_cleaned

,Run,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,M,is_same_charge
0,147115,366639895,58.71410,-7.311320,10.531000,-57.29740,12.82020,-2.202670,2.177660,1,11.28360,-1.032340,-1.88066,-11.077800,2.14537,-2.344030,-2.072810,-1,8.94841,False
1,147115,366704169,6.61188,-4.152130,-0.579855,-5.11278,4.19242,-1.028420,-3.002840,-1,17.14920,-11.713500,5.04474,11.464700,12.75360,0.808077,2.734920,1,15.89300,False
2,147115,367112316,25.54190,-11.480900,2.041680,22.72460,11.66100,1.420480,2.965600,1,15.82030,-1.472800,2.25895,-15.588800,2.69667,-2.455080,2.148570,1,38.38770,True
3,147115,366952149,65.39590,7.512140,11.887100,63.86620,14.06190,2.218380,1.007210,1,25.12730,4.087860,2.59641,24.656300,4.84272,2.330210,0.565865,-1,3.72862,False
4,147115,366523212,61.45040,2.952840,-14.622700,-59.61210,14.91790,-2.093750,-1.371540,-1,13.88710,-0.277757,-2.42560,-13.670800,2.44145,-2.423700,-1.684810,-1,2.74718,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,146511,522575834,12.31310,-10.658000,5.164440,3.36858,11.84330,0.280727,2.690370,-1,1.80181,0.668609,-1.58437,0.537805,1.71967,0.307851,-1.171470,1,8.44779,False
99996,146511,522786431,18.46420,7.854990,15.133000,-7.08659,17.05020,-0.404510,1.092010,1,14.69110,-1.418020,-2.28117,-14.443500,2.68598,-2.383880,-2.126960,1,20.71540,True
99997,146511,522906124,4.18566,-3.273500,-0.308507,-2.59013,3.28801,-0.723075,-3.047630,1,72.81740,-11.074900,-9.28179,-71.369300,14.45010,-2.300410,-2.444050,-1,12.71350,False
99998,146511,523243830,54.46220,11.352600,11.880900,51.92400,16.43280,1.867800,0.808132,-1,8.58671,0.378009,3.07828,8.007050,3.10141,1.677170,1.448610,1,4.69670,False


In [23]:
#df_cleaned.to_csv("data/dielectron_cleaned.csv", index=False)

In [24]:
# df_raw["prueba_e1"] = (df_raw["px1"]**2 + df_raw["py1"]**2 + df_raw["pz1"]**2)**(1/2)

In [25]:
# df_raw[df_raw["prueba_e1"].between(df_raw["E1"]*0.9999, df_raw["E1"]*1.0001)]

In [26]:
# df_raw["prueba_e2"] = (df_raw["px2"]**2 + df_raw["py2"]**2 + df_raw["pz2"]**2)**(1/2)

In [27]:
# df_raw[df_raw["prueba_e2"].between(df_raw["E2"]*0.9999, df_raw["E2"]*1.0001)]

In [28]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(15, 12))
# sns.heatmap(df_raw.corr(), cmap="Greens", annot=True)